In [1]:
import pandas as pd
from sklearn.cluster import KMeans

df = pd.read_csv('../데이터/VIP_churnscoreO_11460_826.csv')

/var/folders/3p/8hc9c44s38j902ytq253kt6m0000gn/T/ipykernel_1545/3789481576.py:4: DtypeWarning: Columns (339) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../데이터/VIP_churnscoreO_11460_826.csv')


,기준년월,발급회원번호,증감율_이용건수_신용_전월,증감율_이용건수_신판_전월,증감율_이용건수_일시불_전월,증감율_이용건수_할부_전월,증감율_이용건수_CA_전월,증감율_이용건수_체크_전월,증감율_이용건수_카드론_전월,증감율_이용금액_신용_전월,...,임직원면제카드수_B0M,우수회원면제카드수_B0M,기타면제카드수_B0M,카드신청건수,Life_Stage,최종카드발급경과월,Slope_Spend,Slope_Balance,Slope_Count,Churn_Score
0,201807,SYN_0,0.454549,0.454549,0.364728,2.0,0.0,0.000000,0.0,0.409767,...,0,0,0,0,5.자녀성장기(2),22,-11116.685714,7214.657143,-0.571429,4.2
1,201807,SYN_1000,0.204653,0.204653,0.204653,0.0,0.0,0.000000,0.0,1.411954,...,0,0,0,0,4.자녀성장기(1),45,-15564.771429,-17170.742857,-2.428571,13.5
2,201807,SYN_1005,0.148929,0.148929,0.099295,2.0,0.0,-0.019939,0.0,0.357926,...,0,0,0,1,2.가족형성기,8,-65200.657143,-10257.428571,-0.057143,11.4
3,201807,SYN_1007,-0.189322,-0.189322,-0.189322,0.0,0.0,0.000000,0.0,0.189824,...,0,0,0,0,3.자녀출산기,0,-3197.514286,-52205.085714,-1.000000,7.2
4,201807,SYN_1009,0.040080,0.040080,0.040080,0.0,0.0,-0.243847,0.0,0.204423,...,0,0,0,0,5.자녀성장기(2),22,4787.828571,-44019.000000,0.657143,2.1


In [5]:
X = df[['Churn_Score']]

# KMeans 모델 생성 (군집 3개)
# random_state를 고정하면 돌릴 때마다 똑같은 결과가 나와서 좋아.
kmeans = KMeans(n_clusters=3, random_state=42)

# 학습 및 예측 (군집 번호 0, 1, 2 생성)
df['cluster_label'] = kmeans.fit_predict(X)

# 각 군집별로 데이터가 몇 개씩 들어갔는지 확인해볼까?
print("\n군집별 데이터 개수:")
print(df['cluster_label'].value_counts().sort_index())


군집별 데이터 개수:
cluster_label
0    3588
1    6630
2    1242
Name: count, dtype: int64


In [7]:
# Target 변수 생성 함수 정의
def create_target(label):
    if label == 0:
        return 1  # 이탈 (군집 0)
    else:
        return 0  # 잔존 (군집 1, 2)

# apply 함수를 써서 Target 컬럼 만들기
df['Target'] = df['cluster_label'].apply(create_target)

# 잘 만들어졌는지 결과 확인!
print("\n생성된 Target 변수 분포:")
print(df['Target'].value_counts())

# 최종 데이터 확인
display(df[['Churn_Score', 'cluster_label', 'Target']].head(10))


생성된 Target 변수 분포:
Target
0    7872
1    3588
Name: count, dtype: int64


,Churn_Score,cluster_label,Target
0,4.2,1,0
1,13.5,0,1
2,11.4,0,1
3,7.2,1,0
4,2.1,1,0
5,2.6,1,0
6,7.7,1,0
7,8.9,0,1
8,4.7,1,0
9,13.8,0,1


In [8]:
import numpy as np

# 1. 랜덤하게 2,000개 뽑기 (df는 이미 있다고 가정)
sampled_df = df.sample(n=2000, random_state=42)

# 2. '발급회원번호' 리스트 추출
id_list = sampled_df['발급회원번호'].tolist()

# 3. 텍스트 파일(member_ids.txt)로 저장하기
with open('member_ids.txt', 'w') as f:
    for member_id in id_list:
        f.write(f"{member_id}\n")

print(f"텍스트 파일 저장 완료! (총 {len(id_list)}개)")
print("첫 5개 ID 미리보기:", id_list[:5])

텍스트 파일 저장 완료! (총 2000개)
첫 5개 ID 미리보기: ['SYN_6135', 'SYN_6441', 'SYN_2343', 'SYN_7742', 'SYN_2164']


In [ ]:
import xgboost as xgb
import shap
import pandas as pd
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor

# --- [1. 데이터 준비] ---
# 분석에 불필요한 컬럼 제거 (ID, 날짜, 타겟, 군집 등)
cols_to_drop = ['발급회원번호', '기준년월', 'Target', 'cluster_label', 'Churn_Score']
# 실제 데이터프레임에 있는 컬럼만 골라서 드롭 (에러 방지)
drop_cols = [c for c in cols_to_drop if c in sampled_df.columns]

X = sampled_df.drop(columns=drop_cols)
X = X.select_dtypes(include=[np.number]) # 숫자형 데이터만 선택
y = sampled_df['Target']

# --- [2. XGBoost 모델 학습] ---
model = xgb.XGBClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=5,
    random_state=42,
    use_label_encoder=False,
    eval_metric='logloss'
)
model.fit(X, y)
print("XGBoost 모델 학습 완료!")

# --- [3. SHAP 중요도(Mean |SHAP|) 계산] ---
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X)

# 각 변수별 평균 절대값(Mean Absolute SHAP) 계산 -> 변수 중요도
shap_importance = np.abs(shap_values).mean(axis=0)

# --- [4. VIF 지수 계산] ---
# 결측치는 평균으로 채워서 계산 (VIF 오류 방지)
X_vif = X.fillna(X.mean())

# 모든 컬럼의 VIF 계산
vif_list = [variance_inflation_factor(X_vif.values, i) for i in range(len(X_vif.columns))]

# --- [5. 데이터프레임 통합 및 정렬] ---
result_df = pd.DataFrame({
    'feature': X.columns,
    'shap_importance': shap_importance,
    'vif': vif_list
})

# SHAP 중요도 기준으로 내림차순 정렬
result_df = result_df.sort_values(by='shap_importance', ascending=False)

# --- [6. CSV 파일 저장] ---
filename = 'feature_importance_shap_vif.csv'
result_df.to_csv(filename, index=False)

print(f"\n[완료] '{filename}' 파일이 생성되었습니다!")
print(f"총 {len(result_df)}개의 변수가 저장되었습니다.")
print("\n[상위 5개 변수 미리보기]")
display(result_df.head())

/usr/local/Caskroom/miniconda/base/envs/myenv/lib/python3.13/site-packages/xgboost/training.py:199: UserWarning: [10:35:46] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1764148644238/work/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBoost 모델 학습 완료!


/usr/local/Caskroom/miniconda/base/envs/myenv/lib/python3.13/site-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)
/usr/local/Caskroom/miniconda/base/envs/myenv/lib/python3.13/site-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.centered_tss
